## Title
[BART: Denoising Sequence-to-sequence Pre-training for Natural Language Generation, Translation, and Comprehension](https://arxiv.org/abs/1910.13461)

## Authors and Year
Mike Lewis, Yinhan Liu, Naman Goyla, Marjan Ghazvininejad, Abdelrahman Mohamed, Omer Levy, Ves Stoyanov, Luke Zettlemoyer (2019)

## Abstract
We present BART, a denoising autoencoder for pretraining sequence-to-sequence models. BART is trained by (1) corrupting text with an arbitrary noising function, and (2) learning a model to reconstruct the original text. It uses a standard Tranformer-based neural machine translation architecture which, despite its simplicity, can be seen as generalizing BERT (due to the bidirectional encoder), GPT (with the left-to-right decoder), and many other more recent pretraining schemes. We evaluate a number of noising approaches, finding the best performance by both randomly shuffling the order of the original sentences and using a novel in-filling scheme, where spans of text are replaced with a single mask token. BART is particularly effective when fine tuned for text generation but also works well for comprehension tasks. It matches the performance of RoBERTa with comparable training resources on GLUE and SQuAD, achieves new stateof-the-art results on a range of abstractive dialogue, question answering, and summarization tasks, with gains of up to 6 ROUGE. BART also provides a 1.1 BLEU increase over a back-translation system for machine translation, with only target language pretraining. We also report ablation experiments that replicate other pretraining schemes within the BART framework, to better measure which factors
most influence end-task performance.

## Model Type
Transformer (BERT encoder + GPT decoder)  

<p align="center">
    <img src="Images/BART/Fusion.jpg" alt="drawing" width="600"/>


## Introduction
#### Pretraining

(1) Text is **corrupted** with an **arbitrary noising function**.   
(2) A sequence-to-sequence model is learned to **reconstruct the original text**.   
   
- **Key advantage** : Noising flexibility
    - Token masking / Token deletion / Text infilling / Sentence permutation / Document rotation
   
#### Fine tuning
Particularly effective when fine tuned for **text generation**
- But also works well for comprehension tasks.
- **Machine translation** -> new scheme; BART model is **stacked above a few additional transformer layers**


## Model
#### Architecture

<p align="center">
    <img src="Images/BART/architecture.jpg" alt="drawing" width="800"/>

전형적인 sequence-to-sequence transformer architecture를 사용함.
- GPT와 마찬가지로 활성화 함수로 ReLU 대신 [GeLU](https://arxiv.org/pdf/1606.08415.pdf) 사용.
- Initialise parameters from $\mathcal{N}$(0,0.02).
- Layers
    - Base model : 6 layers in the encoder and decoder.
    - Large model : 12 layers in each.
   
cf) Why GeLU(Gaussian Error Linear Unit; 2016)?

<p align="center">
    <img src="Images/BART/GELU.jpg" alt="drawing" width="400"/>

- 모든 점에서 미분 가능
- 단조증가함수가 아님 -> 더욱 복잡한 전체 함수를 모델링
- ReLU는 x의 부호에 대해서 gating 되는 것과 달리 GeLU는 x가 다른 입력에 비해서 얼마나 큰지에 대한 비율로 gating됨 -> 확률적 해석 가능   
-> NLP, Vision 등 다양한 task에서 GeLU가 ReLU, ELU에 비해 성능이 좋았다 고 합니다!
   
cf2) BERT와의 차이점
- Decoder의 각 layer가 encoder의 final hidden layer와 cross-attention을 이루고 있음.
- BERT는 word prediction을 위해 추가적인 feed-forward network를 사용하지만 BART는 그렇지 않음.
   
#### Pre-training BART

학습은 original document를 의도적으로 **corruption**시킨 후, reconstruction loss를 optimizing 함.
- Decoder의 output과 original document 간의 cross-entropy를 loss로 설정하여 이를 최소화.
    - 기존의 denoising autoencoder는 특정 noising scheme으로만 training된 반면, BART는 *any type of document corruption* 에 적용 가능.

<p align="center">
    <img src="Images/BART/BART_pretraining.jpg" alt="drawing" width="800"/>

Text corruption을 위해 5가지 transformation 기법을 사용.
- Token masking : random sampling된 token을 [MASK] token으로 치환. BERT에서 사용한 기법.
- Token deletion : random sampling된 token이 삭제됨. Token masking과는 다르게 model은 어떤 position이 missing input인지 결정하게 됨.
- Text infilling : text spans($\lambda$=3인 Poisson distribution 길이 만큼의 token)가 random sampling되고 [MASK] token으로 치환. Token masking과는 다르게 model은 각 span이 몇 개의 token으로 이루어졌는지 예측하게 됨.
- Sentence permutation : 문서로부터 하나의 온점(.)으로 끝나는 문장들을 나누어 random한 순서로 배열을 뒤바꿈.
- Document Rotation : 특정 token이 uniformly at random하게 선택되고, document를 선택된 token부터 시작되게 rotation 시킴. Model은 각 document의 시작점을 맞추게 됨.

## Fine-tuning BART
Pre-trained된 BART는 다양한 용도로 사용될 수 있으며, 대표적으로 4가지의 tasks를 소개함.
    
1. Sequence Classification Tasks

<p align="center">
    <img src="Images/BART/BART_seq_classification.jpg" alt="drawing" width="400"/>

Encoder와 decoder에 같은 input을 주고 **decoder의 final hidden state가 새로운 linear classifier로 전달**됨.
    
2. Token Classification Tasks
전체 document를 encoder와 decoder에 입력하고, **decoder의 top hidden state를 각 단어의 representation으로** 사용함. 이 representation을 token classification에 사용.
    
3. Sequence Generation Tasks
Autoregressive decoder를 갖고 있어 바로 fine-tuning이 가능하며, encoder에 input이 주어지면 **decoder에서 autoregressive하게 output을 생성함**. Question answering이나 summarization 등에 사용될 수 있음.
    
4. Machine translation

<p align="center">
    <img src="Images/BART/BART_machine_translation.jpg" alt="drawing" width="400"/>

전체 BART model을 machine translation을 위한 pre-trained decoder로 사용하고, **새로운 encoder를 추가**하여 encoder-decoder를 fine-tuning함. 새로운 encoder는 foreign words를 BART가 학습한 언어로 denoising 할 수 있는 입력으로 mapping함.
- 새로운 encoder는 두 단계로 학습.
    - 처음에는 대부분의 BART parameter는 그대로 두고, encoder와 BART의 position embedding, BART encoder의 첫 번째 layer의 self-attention input projection matrix만 학습 시킴.
    - 두 번째 단계에서는 모든 parameter들을 학습시킴.
    - 두 단계 모두 다 BART model의 output의 cross-entropy loss로 backpropagate 함.

## Comparing Pre-training Objectives
#### Comparison Objectives
모델 간 training data, training resources, architectural difference, fine-tuning procedures가 다 달라 정확한 비교가 어려움. 알려진 5가지 model을 기반으로 model을 만들어 정확하게 비교하고자 함.
   
1. Language Model = GPT (left to right transformer model)
2. Permuted Language Model = XLNet
3. Masked Language Model = BERT
4. Multitask Masked Language Model = UniLM
5. Masked Seq-to-seq = MASS
   
- 모든 모델은 1M steps on a combination of books and Wikipedia data로 학습됨.
   
#### Tasks
**SQuAD** : extractive question answering task on Wikipedia paragraphs.
    
**MNLI** : bitext classification task to predict whether one sentence entails another.
    
**ELI5** : a long-form abstractive question answering dataset. Models generate answers conditioned on the concatenation of a question and supporting documents.
    
**XSum** : a news summerization dataset with highly abstractive summaries.
    
**ConvAI2** : a dialogue response generation task conditioned on context and a persona.
    
**CNN/DM** : a news summerization dataset. Summaries here are typically closely related to source sentences.
    
#### Results

<p align="center">
    <img src="Images/BART/BART_results.jpg" alt="drawing" width="600"/>

- **Text infilling**을 pre-training에 사용하였을 때 전반적인 performance가 가장 좋았음.
- 가장 결과가 좋지 않았던 것은 document rotation.


## Large-scale Pre-training Experiments
이전까지는 base model(6 layers for encoder and decoder)로 실험 결과를 본 것이었고, 이후로는 RoBERTa 등 large-scale model과의 비교를 위해 large model(12 layers for encoder and decoder)을 사용함.
    
여기서부터는 특이한 점 없이 각 task에 대해 성능이 좋았다고 설명하고 있어서 적당히 건너뛰었습니다 ㅎㅎ ~~(사실 월드컵 너무 보고싶어서 적당히 만들었습니다 대한민국 만세)~~


<p align="center">
    <img src="Images/BART/Guk-bbong.jpg" alt="drawing" width="600"/>


#### 총평
1. 구 Facebook 현 Meta 너네가 짱이다... 수식이 없어서 읽기는 편했던 것 같습니다. 구체적인 구조에 관심이 있으신 분들께서는 code를 직접 까보시는 것이 좋을 것 같습니다.
2. 솔직히 model 구조가 참신했냐 하면 그건 아닌것 같습니다 ~~(솔직히 Google + OpenAI 그냥 숟가락 얹은게 아닌가...)~~. 다만 직접 다른 논문들의 model을 구현해서 객관적으로 성능 비교하려 했다는 그 시도가 대단하다 생각합니다.
3. BART를 선정한 이유는 현재 Cheminformatics에서 BART 구조를 사용한 모델이 forward/retro reaction prediction 등의 task에서 SOTA를 찍었기 때문입니다. 다음 발표때는 관련된 내용으로 발표 해보도록 하겠습니다.